In [ ]:
import json
import pandas as pd
import numpy as np

## Problem 2
In the case of suppressed intra-zonal transfer, the steady state temperature in each zone can be directly computed

$$
T_k^4 = \frac{\gamma_k(1 - \alpha^{\text{sky}})(1 - \bar{\alpha_k})S_0}{\tau \sigma_B}
$$

In [ ]:
def read_zone_parameters(param, params, n):
    return np.array([params['Zone']['{}'.format(i)][param] for i in range(1, n + 1)])

with open('parameters.json') as f:
    params = json.load(f)

n = 6 # Number of zones
gamma = read_zone_parameters('Geometric Factor', params, n)
frac_l = read_zone_parameters('Land Fraction', params, n)
frac_o = read_zone_parameters('Ocean Fraction', params, n)
frac_i = read_zone_parameters('Ice Fraction', params, n)
alpha_l = params['Land Albedo']
alpha_o = params['Ocean Albedo']
alpha_i = params['Ice Albedo']
alpha_s = params['Atmospheric Albedo']
tau = params['Earth Total Emissivity']
sigma = params['Stefan-Boltzmann Constant']
S0 = params['Solar Constant']

# Albedo for the zone is the fractional percent weighted average of
# land, ocean and ice albedos
alpha = frac_l * alpha_l + frac_o * alpha_o + frac_i * alpha_i

T = ((gamma*(1 - alpha_s)*(1 - alpha)*S0)/(tau*sigma))**(0.25)
for i in range(n):
    print('Teq Zone {}: {:.2f} K'.format(i, T[i]))